In [1]:
import numpy as np
import pandas as pd
import ast
import requests
import json
import plotly.express as px
import re
import nltk


In [43]:
# Read in reviews and restaurants datatset 
reviews_df = pd.read_csv('./data/reviews.csv')
restaurants_df = pd.read_csv('./data/yelp_restaurants.csv')

# De-duplicate restaurant DataFrame based on google_id
restaurants_df = restaurants_df.drop_duplicates(subset='google_id', keep='first')

# Make list from list representation in address column
restaurants_df['address'] = restaurants_df['address'].apply(lambda x : ast.literal_eval(x))

In [3]:
restaurants_df.head(2)

,yelp_id,google_id,yelp_name,google_name,address,categories,price_level,avg_rating,review_count,photo_urls,latitude,longitude
0,6XKiHruNrTO8jDAcHLoR3w,ChIJF5m2rSLFmUcRH9sBPHwx3kM,Bäckerhaus Veit Café,Bäckerhaus Veit Café,"[Epplestr. 1 b, 70597 Stuttgart, Germany]","['Bakeries', 'Cafes']",NaN,1.5,5,['https://s3-media2.fl.yelpcdn.com/bphoto/M0vp...,48.749109,9.169810
1,0Ep-rCF4UVrz3ZJkd_gAXQ,ChIJOWEUrcvEmUcRqRXjZal8AZY,Prince of India,Prince of India,"[Urbanstr. 38, 70182 Stuttgart, Germany]",['Indian'],€€,3.0,27,['https://s3-media4.fl.yelpcdn.com/bphoto/rwxy...,48.778664,9.187046


In [4]:
reviews_df.head(2)

,restaurant_id,name,address,reviewer_hash,rating,text,datetime,language,review_count,average_rating,source
0,ChIJOWEUrcvEmUcRqRXjZal8AZY,Prince of India,"Urbanstraße 38, 70182 Stuttgart, Alemania",9028884134327808,5,Sehr gutes Preis-Leistungsverhältnis - sehr le...,2020-10-25T12:58:00Z,de,453,4.1,Google
1,ChIJOWEUrcvEmUcRqRXjZal8AZY,Prince of India,"Urbanstraße 38, 70182 Stuttgart, Alemania",-6024788735085870878,5,Wir waren lange auf der Suche nach einem guten...,2020-10-11T14:30:00Z,de,453,4.1,Google


### Extracting Features based on Keyword Matching using Term Frequency

First we try the most simple approach to extracting restaurant features from reviews. We count occurences (exact matches) of specific keywords such as 'authentisch', 'exzellent', 'wein' which can be used to characterize restaurants. With this simple approach, we get plausible results, but we ignored several problems:
- Word forms and inflections
- Synonym and terms with similar meanding, which we might want to group into one common category
- Negation of keywirds (explicit or implicit)
- Also, the keyword matching approach gives us scores only for a set of pre-defined keywords.


In [5]:
# Build dict with restaurant_ids as keys and list of reviews as values
reviews_dict = dict()
for id in reviews_df.restaurant_id.unique():
    reviews = reviews_df[reviews_df['restaurant_id']==id]['text']
    review_list = []
    for r in reviews:
        if isinstance(r, str):
            review_list.append(r.lower())
    reviews_dict[id] = review_list

In [6]:
# Number of matches / number of reviews
def get_number_of_matches(text, match_string, negative_string=''):
    '''
    Returns the number of exact matches of match_string in text, using the string count() method.
    If a negative_sting is defined, subtracts the occurrences of negative_string
    '''
    if isinstance(text, str):
        count = text.count(match_string)
        if negative_string != '':
            count -= text.count(negative_string)
        return count
    else:
        return 0


In [7]:
# "authentisch", "exzellent", "Wein"
word1 = 'authentisch'
word2 = 'exzellent'
word3 = 'wein'
keyword_list = [word1, word2, word3]

keyword_dict = dict() # Dict to build new DataFrame from
for w in keyword_list:
    word_count_list = []
    for id, reviews in reviews_dict.items():
        count = 0
        for review in reviews:
            count += get_number_of_matches(review, w)
        if (len(reviews)!=0):
            word_count_list.append(count / len(reviews))
        else:
            word_count_list.append(0)
    keyword_dict[w] = word_count_list


In [8]:
# Create new DataFrame for restaurants with keywords, with google_id as first column
restaurant_with_keywords = pd.DataFrame({'google_id' : reviews_df.restaurant_id.unique()})

# Merge with restaurant_df to get the restaurant_names
restaurant_with_keywords = restaurant_with_keywords.merge(restaurants_df, on='google_id', how='left')[['google_id', 'google_name']]

# Iterate through keyword dict and add keywords as new columns for each restaurant
for key, values in keyword_dict.items():
    restaurant_with_keywords[key + '_tf'] = values

In [9]:
restaurant_with_keywords.describe()

,authentisch_tf,exzellent_tf,wein_tf
count,687.000000,687.000000,687.000000
mean,0.015840,0.005573,0.038090
std,0.037564,0.013219,0.080034
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.020000,0.000000,0.040000
max,0.340000,0.086957,0.636364


In [10]:
restaurant_with_keywords.sort_values(by='wein_tf', ascending=False)[['google_name', 'wein_tf']]

,google_name,wein_tf
82,Fresko - Cafe & Restaurant,0.636364
165,Augustenstüble,0.612245
348,Weinstube Zaiß,0.571429
193,Pastis,0.520833
86,Sonnen-Besen Stuttgart-Obertürkheim,0.500000
...,...,...
262,Biddy Earlys Irish Pub,0.000000
258,AS Stern Kebap & Pizza,0.000000
255,Sodexo Betriebsrestaurant T-Systems Leinfelden,0.000000
254,Saray Kebab,0.000000


### Calculating tf-tid scores with SpaCy and sklearn

Next we calculate **tf-idf scores** for terms occuring in our overall collection of reviews. This gives us a score or weight for any term, not just for some pre-defined terms as in the **Keyword Matching** approach.

When assigning tf-idf scores, we want to group together the inflected forms of a term, so we apply lemmatization. Also, we remove punctuation, stop words and normalize all terms to lower case.

The the tf-idf approach assigns a relevancy score for every term, but it does not tell which terms are practical relevant for our application in the restaurant domain. E.g. 'lecker' less relevant than 'service', 'panorama', 'exzellent'. To determine the most suitable terms to characterize restaurants, we still need to rely on our world knowledge about restaurants and what people might like or dislike. 

Based on the tf-idf values, we also calculate a **normalized tf-idf value** for each restaurant, considering the number of reviews. This normalization reduces the effect that restaurants with few and short reviews can achieve a rather high tf-idf score.

The following steps are performed:
- Step 1: Prepare the corpus, consisting of all available reviews.
- Step 2: Learn vocabulary and document-term-matrix for our corpus (lemmatized and tokenized)
- Step 3: Transform document-term-matrix into a tf-idf representation. This gives us the idf scores based on our corpus.
- Step 4: Compute tf-idf weights for specific documents, i.e. for the combined reviews document per restaurant

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
#from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import spacy
import string
punctuations = string.punctuation
from string import punctuation
nlp = spacy.load('de_core_news_sm')


In [12]:
def build_reviews_dict(reviews_df):
    '''
    Returns a dict with restaurants ids as keys and a text concatenating all corresponding reviews as value.
    '''
    reviews_mix_dict = dict()
    restaurant_ids = reviews_df['restaurant_id'].unique()
    for rest_id in restaurant_ids:
        sub_df = reviews_df[reviews_df['restaurant_id']==rest_id]
        # Remove empty text and nans
        sub_df = sub_df[(sub_df['text'] != '')]
        sub_df = sub_df.dropna(subset=['text'])
        reviews_text = ' '.join(sub_df['text'])
        reviews_mix_dict[rest_id] = reviews_text
    return reviews_mix_dict


In [13]:

def tokenize_and_lemmatize(text, canonicals_df=None):
    '''
    Returns a list of tokens for the given text.
    
    The token list is recieved by removing punctuation, lemmatizing and normalizing the text using SpaCy, 
    and removing stop words.
    
    If a DataFrame canonicals_df with canonicals is given, the literals are replaced by their canonical category, 
    specified by the 'literal' respectively 'canonical_category' column.
    This is done after lemmatizing, i.e. in canonicals_df the different word forms need not be considered.

    '''
    text = text.replace('\n', '')
    text = re.sub(r'[^a-zA-Z0-9äöüÄÖÜß]', ' ', text)
    
    doc = nlp(text)
    if (canonicals_df is not None):
        synonyms_dict = pd.Series(canonicals_df['canonical_category'].values, index=canonicals_df['literal']).to_dict()
        # Lemmatize and replace synonyms with canonical categories
        tokens = [token.lemma_.lower() 
                  if token.lemma_ not in synonyms_dict.keys() else synonyms_dict[token.lemma_].lower() 
                  for token in doc]
    else:
        # Lemmatize only
        tokens = [token.lemma_.lower() for token in doc]
        
    # Removing stop words and empty lemmas
    tokens = [token for token in tokens 
              if token not in stop_words 
              and token not in punctuations and token != '' and token !=' ' and token !='  ' and token != '   ']
    return tokens

In [14]:
# Step 1: Prepare corpus, consisting of all available reviews 

reviews_mix_dict = build_reviews_dict(reviews_df)
corpus = reviews_mix_dict.values()

In [15]:
# Step 2: Learn vocabulary and document-term-matrix for our corpus (lemmatized and tokenized)

# Initialize count vectorizer object -> returns document-term matrix, indicating the term frequencies tf for each document
cv = CountVectorizer(tokenizer=tokenize_and_lemmatize)
doc_term_matrix = cv.fit_transform(corpus) # Learn the vocabulary dictionary and return document-term matrix.
doc_term_matrix.shape # 687 documents (restaurants), 30522 tokens (words)


(687, 30522)

In [16]:
# Step 3: Transform document-term-matrix into a tf-idf representation. This gives us the idf scores based on our corpus.
 
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True) 
tf_idf_vector = tfidf_transformer.fit(doc_term_matrix) # Learn the idf vector (global term weights)

# Put idf values in DataFrame and print out  
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(), columns=["idf_weights"]) 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
gut,1.041549
super,1.050681
lecker,1.075451
essen,1.091249
freundlich,1.092842
...,...
hotzenplotz,6.840642
hotz,6.840642
hotwings,6.840642
huh,6.840642


In [17]:
# Show idf_weights for some specific features

feature_list = ['authentisch', 'authentische', 'authentisches', 'authentischen', 
                'exzellent', 'exzellentes', 'exzellente', 'exzellenten', 
                'wein', 'weinkarte', 'weinangebot', 'weinauswahl', 
                'service', 'bedienung', 'kellner', 'kellnerin', 'personal', 'team', 'chef', 'chefin']

df_idf[df_idf.index.isin(feature_list)]

,idf_weights
authentisch,2.240484
bedienung,1.170761
chef,2.131111
chefin,3.022929
exzellent,2.635949
kellner,2.082750
kellnerin,3.033979
personal,1.126909
service,1.107300
team,2.108839


Now that we have the `idf` weights for our corpus (consisting of all restaurant reviews), we can compute the individual `tf-idf` values for any given document.
So next we will compute the `tf-idf` values restaurant-wise, i.e. for the combined review string of each restaurant, to achieve the relevant keywords and `tfidf` values for each restaurant.

In [18]:
# Step 4: Compute tf-idf weights for specific documents, i.e. for the combined reviews document per restaurant

# Calculate count matrix on corpus 
word_count_matrix_testset = cv.transform(corpus)

# Calculate tf-idf scores 
tf_idf_matrix = tfidf_transformer.transform(word_count_matrix_testset)

In [19]:
# Have a look at the tfidf values for the first documents 
first_document_vector = tf_idf_matrix[0] 
feature_names = cv.get_feature_names() 
df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df_tfidf.sort_values(by=["tfidf"], ascending=False)

,tfidf
indisch,0.563353
essen,0.341864
gut,0.201992
lecker,0.184501
freundlich,0.122273
...,...
genieß,0.000000
genieten,0.000000
geniessen,0.000000
geniessbar,0.000000


In [20]:
def top_n_for_feature(feature, count_vectorizer, tf_idf_matrix, reviews_df, restaurants_df=restaurants_df, n=5):
    '''
    Returns a DataFrame with the n largest tfidf scores for the given feature, 
    calculated based on the given CountVectorizer and the given tfidf matrix of shape n_restaurants x n_features
    
    '''
    # Get index of given feature in cv features
    idx = count_vectorizer.get_feature_names().index(feature)
    # Get indeces of corresponding restaurants with n highest tfidf value for this feature
    row = np.array(tf_idf_matrix.todense().T[idx])
    row_df = pd.DataFrame(row.T, columns=[feature + '_tfidf'])
    top_df = row_df.nlargest(n, columns=[feature + '_tfidf'])
    rest_indeces = top_df.index.values
    # Append restaurant ids as column
    rest_ids = []
    restaurant_ids = reviews_df['restaurant_id'].unique()
    for idx in rest_indeces:
        rest_ids.append(restaurant_ids[idx])
    top_df['google_id'] = rest_ids
    # Get restaurant names by merging with restaurant_df
    top_df = top_df.merge(restaurants_df, on='google_id', how='left')[['google_id', 'google_name', feature + '_tfidf']]
    return top_df

In [21]:
def create_augmented_restaurants_df(feature_list, count_vectorizer, tf_idf_matrix, reviews_df, restaurants_df=restaurants_df):
    '''
    Returns the restaurants_df DataFrame, augmented with the tf_idf values and normalized tf_idf values for the given features 
    from feature_list. Thus for each feature two additional columns will be added:
    - {feature}_tf_idf: the tf_idf score for the corresponding restaurant
    - {feature}_norm: tf_idf score normalized by number of reviews (i.e. multiplied with log(1 + |reviews|))
    '''  
    # For each feature, get the corresponding row
    rows = []
    for feature in feature_list:
        # Get index of given feature in cv features
        idx = count_vectorizer.get_feature_names().index(feature)
        # Get corresponding row from tf_idf_matrix
        row = np.array(tf_idf_matrix.todense().T[idx])
        row_df = pd.DataFrame(row.T, columns=[feature + '_tfidf'])
        rows.append(row_df) 
        
    # Create new DataFrame and add columns
    restaurant_ids = reviews_df['restaurant_id'].unique()
    restaurant_features_df = pd.DataFrame(restaurant_ids, columns=['google_id'])
    for i in range(len(feature_list)):
        feature = feature_list[i]
        col = rows[i]
        restaurant_features_df[feature + '_tfidf'] = col
        #restaurant_features_df[feature + '_norm'] =  
        
    # Merge the new DataFrame to restaurants_df and return the result
    augmented_restaurants_df = restaurants_df.merge(restaurant_features_df, on='google_id')
    
    # Add column with normalized tf-idf score (by number of reviews)
    for feature in feature_list:
        augmented_restaurants_df[feature + '_norm'] = augmented_restaurants_df[feature + '_tfidf'] * np.log(1 + augmented_restaurants_df['review_count'])
    
    return augmented_restaurants_df


In [22]:
# Check top n results for some specific features
top_n_for_feature('authentisch', cv, tf_idf_matrix, reviews_df)
top_n_for_feature('exzellent', cv, tf_idf_matrix, reviews_df)
top_n_for_feature('panorama', cv, tf_idf_matrix, reviews_df)
top_n_for_feature('biergarten', cv, tf_idf_matrix, reviews_df)

,google_id,google_name,biergarten_tfidf
0,ChIJ8x4WJ1vbmUcRMl9m_rMfaw0,Tschechen & Söhne,0.462160
1,ChIJI5FW38LcmUcRO0qLJIlDHNg,Cafe Restaurant anno 1897,0.415532
2,ChIJOze-n-jEmUcRIO5IaKjfFfM,Biergarten Stuttgart Klingenbach,0.411324
3,ChIJ28Rpc_rbmUcRhMPmS23lbvw,Biergarten Höhenberg,0.406449
4,ChIJc3Rt-TzFmUcRqU4OrSIcoHE,Neckarbiergarten,0.344290


In [23]:
features= ['authentisch', 'exzellent', 'service', 'panorama', 'biergarten', 'sushi']
restaurants_df_augmented = create_augmented_restaurants_df(features, cv, tf_idf_matrix, reviews_df)

### Tf-idf scores based on preprocessed text (synonym and negation handling)

There are still two issues to be optimized:
- We want to group not only by inflection forms, but also by synonyms and similar terms. For example, German wordings like 'Weinkarte', 'Weinauswahl', 'Weinangebot' all shall fall into the category 'Wein'. Similar, wordings like 'bedienung', 'kellner', 'kellnerin', 'personal', 'team', 'chef', 'chefin' shall all be mapped to a canonical 'service'. 
- Occurences where terms are mentioned in a negated form shall not be included in our scoring.

Thus, we extend the `tf-idf` approach with some preprocessing to deal with synonyms and negation handling:

- To deal with **synonyms**, we simply replace certain wordings with a **canonical form** in a preprocessing step. 
- Also, we want to track keywords only if it was mentioned in a positive sense ('authenisches Essen' vs. 'Essen war nicht gerade authentisch'). If a keyword occurs in a negative rating, we cannot be sure that the keyword refers to the restaurant in a positive way, so we consider only **positive reviews** with rating 4 or 5 in order to deal with potential **negations**.

In [24]:
# Synonym handling: replace certain synonyms with a canonical form
# Requires a mapping from the synonyms to replace to their canonical form.
canonicals_df = pd.read_csv('./data/canonicals.csv', sep=';')
canonicals_df['literal'] = canonicals_df['literal'].str.lower()
canonicals_df['canonical_category'] = canonicals_df['canonical_category'].str.lower()

In [25]:
# Negation handling: 
# To make sure that our extracted keywords actually refer to the corresponding restaurant in a positive way,
# we only consider reviews which rating 4 or 5.
reviews_pos_df = reviews_df[reviews_df['rating']>3]

In [26]:
# Step 1: Prepare corpus, consisting of all positive reviews 
reviews_mix_dict_pos = build_reviews_dict(reviews_pos_df)
corpus = reviews_mix_dict_pos.values()

In [27]:
# Step 2: Learn vocabulary and document-term-matrix for our corpus (lemmatized and tokenized).
# Handle synonyms be replacing according to canonicals_df

# Initialize count vectorizer object -> returns document-term matrix, indicating the term frequencies tf for each document
args = {"canonicals_df": canonicals_df}
cv_2 = CountVectorizer(tokenizer=lambda text: tokenize_and_lemmatize(text, **args))
doc_term_matrix_2 = cv_2.fit_transform(corpus) # Learn the vocabulary dictionary and return document-term matrix.
doc_term_matrix_2.shape # 684 documents (restaurants), 30522 tokens (words)


(684, 22299)

In [28]:
# Step 3: Transform document-term-matrix into a tf-idf representation. This gives us the idf scores based on our corpus.
 
tfidf_transformer_2 = TfidfTransformer(smooth_idf=True, use_idf=True) 
tf_idf_vector_2 = tfidf_transformer_2.fit(doc_term_matrix_2) # Learn the idf vector (global term weights)

# Put idf values in DataFrame and print out  
df_idf_2 = pd.DataFrame(tfidf_transformer_2.idf_, index=cv_2.get_feature_names(), columns=["idf_weights"]) 
df_idf_2.sort_values(by=['idf_weights'])

,idf_weights
gut,1.041735
service,1.044784
super,1.050909
lecker,1.071081
essen,1.093268
...,...
hähnchenburger,6.836272
hähnchendöner,6.836272
hähnchengeschnetzeltes,6.836272
hähnchenschenkel,6.836272


In [29]:
# Show idf_weights for some specific features

feature_list = ['authentisch', 'authentische', 'authentisches', 'authentischen', 
                'exzellent', 'exzellentes', 'exzellente', 'exzellenten', 
                'wein', 'weinkarte', 'weinangebot', 'weinauswahl', 
                'service', 'bedienung', 'kellner', 'kellnerin', 'personal', 'team', 'chef', 'chefin', 'mitarbeiter', 'mitarbeiterin']

df_idf_2[df_idf_2.index.isin(feature_list)]

,idf_weights
authentisch,2.292977
exzellent,2.646617
service,1.044784
wein,2.387755


In [30]:
# Step 4: Compute tf-idf weights for specific documents, i.e. for the combined reviews document per restaurant

# Calculate count matrix on corpus 
word_count_matrix_testset_2 = cv_2.transform(corpus)

# Calculate tf-idf scores 
tf_idf_matrix_2 = tfidf_transformer_2.transform(word_count_matrix_testset_2)

In [31]:
# Create dataframe with all additional features from features_list
tf_idf_matrix_2

<684x22299 sparse matrix of type '<class 'numpy.float64'>'
	with 171840 stored elements in Compressed Sparse Row format>

In [32]:
# Check top n results for some specific features
top_n_for_feature('authentisch', cv_2, tf_idf_matrix_2, reviews_pos_df)
top_n_for_feature('exzellent', cv_2, tf_idf_matrix_2, reviews_pos_df)
top_n_for_feature('panorama', cv_2, tf_idf_matrix_2, reviews_pos_df)
top_n_for_feature('biergarten', cv_2, tf_idf_matrix_2, reviews_pos_df)

,google_id,google_name,biergarten_tfidf
0,ChIJ8x4WJ1vbmUcRMl9m_rMfaw0,Tschechen & Söhne,0.484799
1,ChIJ28Rpc_rbmUcRhMPmS23lbvw,Biergarten Höhenberg,0.415541
2,ChIJI5FW38LcmUcRO0qLJIlDHNg,Cafe Restaurant anno 1897,0.387325
3,ChIJOze-n-jEmUcRIO5IaKjfFfM,Biergarten Stuttgart Klingenbach,0.368788
4,ChIJc3Rt-TzFmUcRqU4OrSIcoHE,Neckarbiergarten,0.346839


In [33]:
features= ['authentisch', 'exzellent', 'service', 'panorama', 'biergarten', 'sushi']
restaurants_df_augmented_neg_syn = create_augmented_restaurants_df(features, cv_2, tf_idf_matrix_2, reviews_pos_df)
restaurants_df_augmented_neg_syn.to_csv('./data/restaurants_df_augmented.csv')
restaurants_df_augmented_neg_syn.to_excel('./data/restaurants_df_augmented.xlsx')

In [34]:
restaurants_df_augmented.sort_values(by=['service_tfidf', 'avg_rating'], ascending=[False, True])[['google_name', 'service_tfidf']].head(10)


,google_name,service_tfidf
335,Christophorus,0.222951
204,Trattoria Piloni,0.200201
139,Vetter Essen & Trinken,0.192066
678,Oggi,0.184908
191,Restaurant Löwenstube Stuttgart,0.182922
331,Weinstube am Stadtgraben,0.181077
105,Thai-Restaurant Lilavadee,0.179229
74,Restaurant Girasole,0.174921
474,Möhringer Hexle,0.172631
380,Ristorante Pizzeria La Taverna da Pagano,0.170982


In [35]:
restaurants_df_augmented_neg_syn.sort_values(by=['service_tfidf', 'avg_rating'], ascending=[False, True])[['google_name', 'service_tfidf']].head(10)

,google_name,service_tfidf
313,DAS WOLLCAFÉ,0.624443
665,RISTORANTE MANGIA!,0.356071
540,Hummelgraben,0.352896
512,Houstons Fine Burrito Company,0.341112
421,Alte Kanzlei,0.335943
190,Restaurant Löwenstube Stuttgart,0.330079
56,Untere Körschmühle (Gastronomie),0.329548
654,Weinstube Unmüßig,0.328478
247,Restaurant Zur Schreinerei,0.327187
200,"Restaurant ""La Perla"" im Schießsport-Zentrum",0.324270


In [36]:
# Sort by normed score for 'service'
restaurants_df_augmented_neg_syn.sort_values(by=['service_norm'], ascending=[False])[['google_name', 'service_tfidf', 'service_norm', 'review_count', 'avg_rating']].head(10)

,google_name,service_tfidf,service_norm,review_count,avg_rating
421,Alte Kanzlei,0.335943,1.613875,121,3.5
675,Oggi,0.288074,1.293059,88,3.5
113,Taverna Sultan Saray,0.307651,1.238405,55,3.5
493,Brauhaus Schönbuch,0.269448,1.224180,93,3.5
138,Vetter Essen & Trinken,0.311865,1.172988,42,4.5
324,reiskorn,0.240425,1.087153,91,4.0
483,Zur Sattlerei,0.315823,1.084533,30,3.0
63,PILUM,0.288022,1.083307,42,3.5
369,Weinstube Kachelofen,0.304438,1.082384,34,4.0
231,PIER 51 Restaurant & Bar,0.243299,1.080893,84,3.5


In [37]:
restaurants_df_augmented_neg_syn['service_norm'].describe()

count    684.000000
mean       0.347087
std        0.276977
min        0.000000
25%        0.120987
50%        0.285670
75%        0.519578
max        1.613875
Name: service_norm, dtype: float64

### Create dummy variables for categories from 'categories' column

Last, we convert the list string in the categories column to a list and create dummy variables from them. This means, for any value in the `categories` column, a new column will be added with value 1 or 0, indicating whether the specific restaurants belongs to the given category or not. This gives us a vector-like representation of the restaurants which can be applied e.g. for calculating similarities between restaurants.

In [38]:
# Convert string representation of list to list
restaurants_df_augmented_neg_syn['categories'] = restaurants_df_augmented_neg_syn['categories'].apply(lambda x : ast.literal_eval(x))

In [39]:
restaurants_df_augmented_neg_syn.head(2)

,yelp_id,google_id,yelp_name,google_name,address,categories,price_level,avg_rating,review_count,photo_urls,...,service_tfidf,panorama_tfidf,biergarten_tfidf,sushi_tfidf,authentisch_norm,exzellent_norm,service_norm,panorama_norm,biergarten_norm,sushi_norm
0,6XKiHruNrTO8jDAcHLoR3w,ChIJF5m2rSLFmUcRH9sBPHwx3kM,Bäckerhaus Veit Café,Bäckerhaus Veit Café,"[Epplestr. 1 b, 70597 Stuttgart, Germany]","[Bakeries, Cafes]",NaN,1.5,5,['https://s3-media2.fl.yelpcdn.com/bphoto/M0vp...,...,0.105417,0.0,0.0,0.0,0.000000,0.0,0.188881,0.0,0.0,0.0
1,0Ep-rCF4UVrz3ZJkd_gAXQ,ChIJOWEUrcvEmUcRqRXjZal8AZY,Prince of India,Prince of India,"[Urbanstr. 38, 70182 Stuttgart, Germany]",[Indian],€€,3.0,27,['https://s3-media4.fl.yelpcdn.com/bphoto/rwxy...,...,0.274062,0.0,0.0,0.0,0.231261,0.0,0.913231,0.0,0.0,0.0


In [40]:
# Stack and derive dummy variables, so for every category in the list there will be added 
cat_df = pd.get_dummies(restaurants_df_augmented_neg_syn['categories'].apply(pd.Series).stack()).sum(level=0)

In [44]:
# Write to xlsc and csv
restaurants_with_new_categories = restaurants_df_augmented_neg_syn.join(cat_df)
restaurants_with_new_categories.to_excel('./data/restaurants_df_augmented.xlsx')
restaurants_with_new_categories.to_csv('./data/restaurants_df_augmented.csv')